In [29]:
import torch
import torch.nn as nn
import numpy as np
import string
import emoji

In [30]:
def stl(word):
	nlist = np.zeros((4, 27))
	alphabet = list(string.ascii_lowercase)
	if len(word) < 4:
		word += ' '*(4-len(word))
	for i in range(4):
		if word[i] == ' ':
			nlist[i][26] = 1.0
		else:
			alphabet_index = alphabet.index(word[i])
			nlist[i][alphabet_index] = 1.0
	return nlist

In [31]:
emojis = np.identity(7) # 0: hat, 1: rat, 2: cat, 3: flat, 4: matt, 5: cap, 6: son

x_train = torch.tensor([stl('hat '), stl('rat '), stl('cat '), stl('flat'), stl('matt'), stl('cap '), stl('son ')]).float()

y_train = torch.tensor([emojis[0], emojis[1], emojis[2], emojis[3], emojis[4], emojis[5], emojis[6]]).float()

index_to_emoji = ['hat', 'rat', 'cat', 'flat', 'matt', 'cap', 'son']

In [32]:
class LongShortTermMemoryModel(nn.Module):
    def __init__(self):
        super(LongShortTermMemoryModel, self).__init__()

        self.lstm = nn.LSTM(108, 128, batch_first=True)  # 128 is the state size
        self.dense = nn.Linear(128, 7)  # 128 is the state size

    def reset(self):  # Reset states prior to new input sequence
        zero_state = torch.zeros(1, 1, 128).float()  # Shape: (number of layers, batch size, state size)
        self.hidden_state = zero_state
        self.cell_state = zero_state

    def logits(self, x):  # x shape: (sequence length, batch size, encoding size)
        out, (self.hidden_state, self.cell_state) = self.lstm(x, (self.hidden_state, self.cell_state))
        # print(out.size())
        return self.dense(out.reshape(-1, 128))

    def f(self, x):  # x shape: (sequence length, batch size, encoding size)
        return torch.softmax(self.logits(x), dim=1)

    def loss(self, x, y):  # x shape: (sequence length, batch size, encoding size), y shape: (sequence length, encoding size)
        # print(y)
        # print(self.logits(x).size())
        # print(y.argmax(1))
        return nn.functional.cross_entropy(self.logits(x), y.argmax(1))


In [33]:
model = LongShortTermMemoryModel()

optimizer = torch.optim.RMSprop(model.parameters(), lr=0.001)

In [34]:
for epoch in range(500):
    for batch in range(7):
        model.reset()
        model.loss(x_train[batch].reshape(1,1,-1), y_train[batch].reshape(1,-1)).backward()
        optimizer.step()
        optimizer.zero_grad()


In [35]:
for index in range(7):
	y = torch.tensor([stl(index_to_emoji[index])]).reshape(1,1,-1).float()
	print(emoji.emojize(f'Input: {index_to_emoji[index]}, Predicted: :{index_to_emoji[model.f(y).argmax(1)]}:'))
index = torch.tensor([stl('rt')]).reshape(1,1,-1).float()
print(emoji.emojize(f'Input: rt, Predicted: :{index_to_emoji[model.f(index).argmax(1)]}:'))


Input: hat, Predicted: :hat:
Input: rat, Predicted: 🐀
Input: cat, Predicted: 🐈
Input: flat, Predicted: :flat:
Input: matt, Predicted: :matt:
Input: cap, Predicted: :cap:
Input: son, Predicted: :son:
Input: rt, Predicted: 🐀


In [39]:
rat = "rat"
print("\N{hat}")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 0-6: unknown Unicode character name (3406385143.py, line 2)